In [ ]:
from transformers import GPT2LMHeadModel

In [ ]:
model_hf = GPT2LMHeadModel.from_pretrained("gpt2") # 124M
sd_hf = model_hf.state_dict()

for k, v in sd_hf.items():
    print(k, v.shape)

In [ ]:
sd_hf["transformer.wpe.weight"].view(-1)[:20]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(sd_hf["transformer.wpe.weight"], cmap="gray")

In [ ]:
plt.plot(sd_hf["transformer.wpe.weight"][:, 150])
plt.plot(sd_hf["transformer.wpe.weight"][:, 200])
plt.plot(sd_hf["transformer.wpe.weight"][:, 250])

In [ ]:
plt.imshow(sd_hf["transformer.h.1.attn.c_attn.weight"][:300,:300], cmap="gray")

In [ ]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)


In [ ]:
# 我们手动实现文本生成的采样逻辑,通过 PyTorch 进行推理，不依赖 Hugging Face 的 pipeline
import torch
from torch.nn import functional as F

# 加载预训练的 GPT-2 模型
model = GPT2LMHeadModel.from_pretrained("gpt2")  # 124M
model.eval()
model.to('cuda')

# 设置随机种子，确保采样结果可复现
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# 初始输入 token 序列
tokens = [15496, 11, 314, 1101, 257, 3303, 2746, 11]  # 对应编码后的 token
tokens = torch.tensor(tokens, dtype=torch.long)  # 转为张量，形状为 (8,)
tokens = tokens.unsqueeze(0).repeat(5, 1)  # 扩展为 (5, 8)，表示生成 5 个样本
x = tokens.to('cuda')  # 送入 GPU

# 开始生成！
while x.size(1) < 30:  # 最长生成长度为 30
    # 前向传播，获取 logits
    with torch.no_grad():
        logits = model(x)[0]  # 输出形状为 (B, T, vocab_size)
        logits = logits[:, -1, :]  # 只取最后一个位置的 logits，形状为 (B, vocab_size)
        probs = F.softmax(logits, dim=-1)  # 转为概率分布
        # 执行 top-k 采样，k=50（和 Hugging Face 默认一致）
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)  # (B, 50)
        # 从 top-k 中按概率采样一个 token
        # 注意 multinomial 不要求输入概率之和为 1
        ix = torch.multinomial(topk_probs, 1)  # (B, 1)
        # 根据采样结果取出对应的 token 索引
        xcol = torch.gather(topk_indices, -1, ix)  # (B, 1)
        # 将新 token 拼接到输入序列后面
        x = torch.cat((x, xcol), dim=1)

# 输出生成的文本
import tiktoken
enc = tiktoken.get_encoding('gpt2')
for i in range(5):
    tokens = x[i, :30].tolist()  # 取前 30 个 token
    decoded = enc.decode(tokens)  # 解码为字符串
    print(">", decoded)


In [ ]:
# tiny shakespeare dataset
with open('input.txt', 'r') as f:
    text = f.read()
data = text[:1000]
print(data[:100])

In [ ]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode(data)
print(tokens[:24])

In [ ]:
import torch
buf = torch.tensor(tokens[:24 + 1])
x = buf[:-1].view(4, 6)
y = buf[1:].view(4, 6)
print(x)
print(y)

In [ ]:
print(sd_hf["lm_head.weight"].shape)
print(sd_hf["transformer.wte.weight"].shape)

In [ ]:
(sd_hf["lm_head.weight"] == sd_hf["transformer.wte.weight"]).all()

In [ ]:
print(sd_hf["lm_head.weight"].data_ptr())
print(sd_hf["transformer.wte.weight"].data_ptr())

In [ ]:
# 残差流中的标准差会随着层数增加而增长（如果不控制的话）
# 这里通过缩放噪声，使其在多层中保持稳定

x = torch.zeros(768)  # 初始化一个 768 维的张量（模拟残差流）
n = 100  # 比如说 100 层

for i in range(n):
    # 每层加上一组高斯噪声，并使用 1/sqrt(n) 缩放防止方差爆炸
    x += n**-0.5 * torch.randn(768)

# 打印最终 x 的标准差
print(x.std())


In [ ]:
import torch

# 一个非常简单的小型多层感知机（MLP）
net = torch.nn.Sequential(
    torch.nn.Linear(16, 32),  
    torch.nn.GELU(),          
    torch.nn.Linear(32, 1)   
)

torch.random.manual_seed(42)
x = torch.randn(4, 16)
y = torch.randn(4, 1)
# 将模型参数的梯度清零
net.zero_grad()
# 前向传播，获得模型输出
yhat = net(x)
# 计算均方误差损失（MSE），reduction='mean' 表示对所有样本平均
loss = torch.nn.functional.mse_loss(yhat, y)
# 反向传播，计算所有参数的梯度
loss.backward()
# 打印第一个线性层的权重梯度（只显示前10个元素）
print(net[0].weight.grad.view(-1)[:10])


In [ ]:
net.zero_grad()  # 清空所有梯度
for i in range(4):  # 模拟每次处理一个样本（即 B=1）
    yhat = net(x[i])  # 前向传播，预测第 i 个样本
    loss = torch.nn.functional.mse_loss(yhat, y[i])  # 单样本损失（默认是 scalar）
    loss = loss / 4  # 手动除以 4，恢复平均损失的语义
    loss.backward()  # 累积梯度
# 打印第一个线性层的权重梯度（前10个元素）
print(net[0].weight.grad.view(-1)[:10])
